<a href="https://colab.research.google.com/github/HemantGorle/LSTM_Next_Alphabet_Prediction/blob/main/LSTM_to_predict_next_alphabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

In [26]:
# load ascii text  
!git clone https://github.com/HemantGorle/LSTM_Next_Alphabet_Prediction.git
filename = "/content/LSTM_Next_Alphabet_Prediction/Alphabets.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()

fatal: destination path 'LSTM_Next_Alphabet_Prediction' already exists and is not an empty directory.


In [27]:
raw_text = raw_text.replace(" ", "")

In [28]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [29]:
print(char_to_int)

{'\n': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [30]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  27
Total Vocab:  27


In [31]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  26


In [32]:
print(dataX)

[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26]]


In [33]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [34]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [35]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, mode='min')
callbacks_list = [checkpoint]

In [36]:
model.fit(X, y, epochs=700, callbacks=callbacks_list)

Epoch 1/700
1/1 [==============================] - 2s 2s/step - loss: 3.2959

Epoch 00001: saving model to weights-improvement-01.hdf5
Epoch 2/700
1/1 [==============================] - 0s 5ms/step - loss: 3.2950

Epoch 00002: saving model to weights-improvement-02.hdf5
Epoch 3/700
1/1 [==============================] - 0s 6ms/step - loss: 3.2944

Epoch 00003: saving model to weights-improvement-03.hdf5
Epoch 4/700
1/1 [==============================] - 0s 12ms/step - loss: 3.2930

Epoch 00004: saving model to weights-improvement-04.hdf5
Epoch 5/700
1/1 [==============================] - 0s 5ms/step - loss: 3.2931

Epoch 00005: saving model to weights-improvement-05.hdf5
Epoch 6/700
1/1 [==============================] - 0s 5ms/step - loss: 3.2917

Epoch 00006: saving model to weights-improvement-06.hdf5
Epoch 7/700
1/1 [==============================] - 0s 6ms/step - loss: 3.2908

Epoch 00007: saving model to weights-improvement-07.hdf5
Epoch 8/700
1/1 [==============================]

In [37]:
# load the network weights
filename = "weights-improvement-699.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [40]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
val = input("Enter any alphabet from A - Z:\n")
start = char_to_int.get(val.upper())-1
pattern = dataX[start]
#print ("User input is :", userinput)
#print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("predicted next alphabet is: ")
# generate characters
for i in range(1):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nprogram execution is finished.")

Enter any alphabet from A - Z:
 R
predicted next alphabet is: 
S
program execution is finished.
